Import library

In [2]:
from CoffeeLibrary.PreProcess import *
from CoffeeLibrary.FeatureCreate import *

ModuleNotFoundError: No module named 'dataprep'

Load and process data

In [3]:
path = 'coffee_df_with_type_and_region.csv'
prep = PreProcess(path)

prep.clean_origin()
prep.clean_location()
prep.standardize_price()
prep.standardize_agtron()
prep.drop_cols()
prep.drop_nas()

NameError: name 'PreProcess' is not defined

Create Features

In [4]:
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_1').top_n_unigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_2').top_n_unigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_3').top_n_unigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_1').top_n_bigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_3').top_n_bigram(20)], axis=1)
prep.df.drop(['desc_1','desc_2','desc_3'], axis=1, inplace=True)

NameError: name 'pd' is not defined

Split Dataframe

In [5]:
features = df.columns[df.columns != 'rating']
X_train, X_test, y_train, y_test = split(prep.df, features)

NameError: name 'df' is not defined

Train Model and Predict

In [6]:
regressor = train_model(X_train, y_train, features)
results = predict(X_test, y_test, regressor, features)

NameError: name 'train_model' is not defined

Compute test roc_auc metric

In [7]:
roc_auc_score = roc_auc_metric(results)
print(roc_auc_score)

NameError: name 'roc_auc_metric' is not defined